# 1. Database Construction

The code in this notebook describes the construction of the database. Web scraping to retrieve the Product Information files for all drug products containing a small molecule active ingredient listed on the European Medicines Agency central register. The documents were converted to plain text format and split if they contained more than one product. Regular expression matching was used to isolate relevant portions of each document. .csv files requiring manual curation were exported as "document_.csv" and subsequently renamed and read as "document.csv" to prevent accidental overwrite. 

In [ ]:
# Note that all necessary imports are in constructiontools.py
from constructiontools import *

In [ ]:
# Read necessary csv files

raw_output = pd.read_csv("../csv_files/medicines_raw_output.csv", encoding="ansi") #EMA Central Register (as of 2/1/2024) with header removed
ema_substances = pd.read_csv("../csv_files/sms.csv", encoding="ansi") #SMS current export, 2/1/2024

In [ ]:
# Applying a pipe of tools from constructiontools.py

raw_output_cleaned = (
    raw_output
    .pipe(remove_products)
    .pipe(identify_small_molecules, ema_substances)
    .pipe(identify_nucleic_acids, ema_substances)
    .pipe(assign_inchi_key, ema_substances)
)

In [ ]:
# Removing high molecular weight drugs

missing_values = raw_output_cleaned[raw_output_cleaned["InChI_Keys"].apply(lambda x: "not found" in x)]
missing_values.to_csv("../csv_files/missing_values_.csv")
no_missing_exploded = raw_output_cleaned[raw_output_cleaned["InChI_Keys"].apply(lambda x: "not found" not in x)].explode(["Active substance", "InChI_Keys"])
no_missing_exploded["Mol"] = no_missing_exploded["InChI_Keys"].apply(get_inchi_from_inchikey)
no_missing_exploded["MolWt"] = no_missing_exploded["Mol"].apply(get_molwt)
no_missing_exploded.to_csv("../csv_files/highmolwt_.csv")

In [ ]:
# Removing high molecular weight drugs after inspecting highmolwt_.csv

indices_to_drop = [253, 557, 608, 670, 1186, 332, 581, 1992, 1034, 1267, 347, 1944, 301, 1474, 1725, 801, 860, 580, 431, 1485, 810, 931, 646, 890, 693, 1021, 1169, 44, 62, 155, 227, 238, 258, 283, 295, 353, 366, 484, 486, 506, 543, 613, 680, 709, 808, 838, 866, 1037, 1139, 1193, 1211, 1350, 1561, 1597, 1607, 1612, 1663, 1669, 1755, 1816, 1829, 1840, 1863, 1931, 1966, 1987]
raw_output_dropped = raw_output_cleaned.drop(indices_to_drop)

In [ ]:
# Retrieving pdf link from all URLs in raw_output_dropped. Exporting to manually add in PDF URLs not identified.

raw_output_dropped["PDF_URL"] = raw_output_dropped["URL"].apply(get_pdf_link)
raw_output_dropped.to_csv("../csv_files/sourcing_pdfs_.csv")

In [ ]:
# Downloading .pdf files to correct directory, and converting them to .txt. Note that some withdrawn products did not have 
# an available EPAR product information file at the time of analysis. 

raw_output_with_links = pd.read_csv("../csv_files/sourcing_pdfs.csv", index_col=0)
raw_output_with_links = raw_output_with_links[~raw_output_with_links["PDF_URL"].str.contains("Source pdf URL manually")]
download_pdfs(raw_output_with_links)
generate_txt_files(raw_output_with_links)

In [ ]:
# Splitting .txt files that contain more than one SmPC into separate documents and assigning systematic names

pattern = re.compile(r"QUALITATIVE\s*AND\s*QUANTITATIVE")

num_smpcs = []
for index in raw_output_with_links.index:
    with open(f"../regulatory_docs/txt_docs/{index}.txt", encoding="utf-8") as f:
        txt = f.read()
        matches = pattern.findall(txt)
        num_smpcs.append((index, len(matches)))
smpc_counts = pd.DataFrame(num_smpcs, columns=["Doc", "Count"])

#dfnew[dfnew["Count"]==0]  manually converted doc 1490 to .txt format

for index in raw_output_with_links.index:
    with open(f"../regulatory_docs/txt_docs/{index}.txt", "r", encoding="utf-8") as f:
        txt = f.read()
        formatted_text = re.sub(pattern, "/=/=/=QUANTITATIVE AND QUALITATIVE", txt)
        split_text = formatted_text.split("/=/=/=")
        split_text.pop(0)
        for smpc in range(len(split_text)):
            with open(f"./regulatory_docs/txt_docs_split/{index}_{smpc}.txt", "a+", encoding="utf-8") as f:
                f.write(split_text[smpc])

In [ ]:
# Making a new datafrake to account for multiple SmPCs in a single .txt file

split_dataframe = update_master_table(raw_output_with_links)

In [ ]:
# Using regular expressions to add relevant SmPC sections for later manual curation

all_raw_data = (
    split_dataframe
    .pipe(add_pharmaceutical_form)
    .pipe(add_pharmacokinetics)
    .pipe(add_qq_composition)
    .pipe(add_excipients)
)
# Remove dosage froms that are not the highest strength clinically approved product
all_raw_data.to_csv("../csv_files/remove_low_strength_.csv")

In [ ]:
# Add the dose and active ingredients, ensuring it is clear what form of the drug
# the dose refers to

low_strength_removed = pd.read_csv("../csv_files/remove_low_strength.csv", index_col=0)
low_strength_removed = low_strength_removed[low_strength_removed["Delete"] != "Delete"].drop(columns=["Delete"])
low_strength_removed.to_csv("../csv_files/adding_pharmacokinetics_.csv")

In [ ]:
# Using excipient_map, a dataframe which shows each product side-by-side with the updated excipient list is showm.
# It is then exported for manual curation.

register1 = pd.read_csv("../csv_files/adding_pharmacokinetics.csv", index_col=0)
mapping1 = pd.read_csv("../csv_files/adding_excipients.csv", index_col=0) # Previously curated excipient mapping
new_df = clean_excipients(register1, mapping1)
inactive_ingredients = pd.read_csv("../csv_files/excipient_map.csv", encoding="ANSI", index_col=0)
final_prods = pd.merge(register1, inactive_ingredients["Excipients_Controlled"], left_index=True, right_index=True)
final_prods.dropna(subset="Excipients_Controlled", inplace=True)
final_prods["Excipients_Controlled"] = final_prods["Excipients_Controlled"].apply(nan_to_list)
final_df = pd.merge(new_df, final_prods[["Product number", "Route", "Excipients_Controlled"]].drop_duplicates(subset=["Product number", "Route"]), how="left", on=["Product number", "Route"])
final_df.to_csv("../csv_files/fixing_excipients_.csv")

In [ ]:
# Fixing a range of curation errors and grouping certain excipients, e.g. polyethylene oxide
# Note that this process was iterative during the study and that errors in excipient curation were correct on a continuous basis.

excipients_df = pd.read_csv("../csv_files/fixing_excipients.csv", index_col=0)
replacements = {
                "POLYVINYL ALCHOL PART. HYDROLISED": "POLYVINYL ALCOHOL PART. HYDROLISED", 
                "LOW-SUBSTITUTED HYDROXYPROPYL CELLULOSE": "LOW SUBSTITUTED HYDROXYPROPYL CELLULOSE", 
                "HYDROXY PROPYL CELLULOSE (LOW SUBSTITUTED)": "LOW SUBSTITUTED HYDROXYPROPYL CELLULOSE",
                "HYDROXYETHYL CELLULOSE": "HYDROXYETHYLCELLULOSE",
                "AMMONIUM GLYCYRRHIZATE": "AMMONIUM GLYCYRRHIZATE",
                "POLYVINYL ALCOHOL PART. HYDROLISED": "POLYVINYL ALCOHOL PART. HYDROLISED",
                "SACCHARIN SODIUM": "SODIUM SACCHARIN",
                "SILICA HYDROPHOBIC COLLOIDAL": "HYDROPHOBIC COLLOIDAL SILICA",
                "HYRPOMELLOSE": "HYPROMELLOSE",
                "SULPHUR DIOXIDE ": "SULPHUR DIOXIDE",
                "SODIUM CARMELLOSE": "CARMELLOSE SODIUM",
                "SODIUM BICARBONATE (E500)": "SODIUM HYDROGEN CARBONATE",
                "METHYLHYDROXYPROPYLCELLULOSE": "HYPROMELLOSE",
                "POLYETHYLENE OXIDE 7000K": "POLYETHYLENE OXIDE",
                "POLYETHYLENE OXIDE 200K": "POLYETHYLENE OXIDE",
                "METHYLHYDROXYPROPYLCELLULOSE": "HYPROMELLOSE",
                "SILICA DENTAL TYPE": "SILICON DIOXIDE, ANHYDROUS",
                "SILICA COLLOIDAL HYDROPHOBIC": "HYDROPHOBIC COLLOIDAL SILICA",
                "GLYCOL MONOSTEARATE": "GLYCEROL MONOSTEARATE",
                "HYDROXYPROPYL CELLULOSE": "HYDROXYPROPYLCELLULOSE",
                "MONOAMMONIUM GLYCYRRHIZINATE": "AMMONIUM GLYCYRRHIZATE",
                "GLYCERIN": "GLYCEROL",
                "MAIZE MALTODEXTRIN": "MALTODEXTRIN",
                "MALTODEXTRIN ": "MALTODEXTRIN",
                "AMMONIUM GLYCYRRHIZATE": "AMMONIUM GLYCYRRHIZATE"
                }
def modify_dataframe(df, replacements):
    def replace_values(lst):
        modified_lst = []
        for item in lst:
            if item.upper() in ["BLANK", "CELLULOSE"]:
                continue
            else:
                modified_item = replacements.get(item.upper(), item)
                modified_lst.append(modified_item)
        return [item for item in modified_lst if item is not None]
    df["Excipients_Controlled"] = df["Excipients_Controlled"].apply(ast.literal_eval)
    df["Excipients_Controlled"] = df["Excipients_Controlled"].apply(replace_values)
modify_dataframe(excipients_df, replacements)
excipients_df["Excipients_Final"] = excipients_df["Excipients_Controlled"].apply(lambda x: list(set(x)))
excipients_df.to_csv("../csv_files/final_master_df.csv")

In [ ]:
# Generating a precursor to parent_drugs.csv
active_forms = pd.read_csv("../csv_files/fixing_excipients.csv")
active_forms["actives_in_dosage_form"] = active_forms["actives_in_dosage_form"].apply(turn_actives_into_list)
active_forms["actives_in_dosage_form"]
final_actives = active_forms["actives_in_dosage_form"].explode()
final_actives_df = pd.DataFrame(final_actives)
final_actives_df["product"] = final_actives_df.index
final_actives_df.rename(columns = {"actives_in_dosage_form": "active_PSS"}, inplace=True)
final_actives_df_identifiers = assign_identifier_to_actives(final_actives_df, ema_substances)
final_actives_df_identifiers["pss_inchi"] = final_actives_df_identifiers["Inchikey"].apply(get_inchi_from_inchikey)
final_actives_df_identifiers["IsomericSMILES"] = final_actives_df_identifiers["Inchikey"].apply(get_isomeric_smiles_from_inchikey)
final_actives_df_identifiers.to_csv("../csv_files/withinchikeys_.csv")

In [ ]:
# Generating different line notations for active ingredient and getting ChEMBL ID for parent drug.
# Those without will be manually curated from adding_chembl_to_actives_.csv

df4 = pd.read_csv("../csv_files/withinchikeys.csv")
df4["ps_smiles"], df4["ps_num_drugs"] = zip(*df4["IsomericSMILES"].apply(desolvate))
df4["p_smiles"], df4["p_inchi"], df4["p_inchikey"] = *df4["ps_smiles"], df4["ps_num_drugs"] = zip(*df4["IsomericSMILES"].apply(get_p_form))
df4["p_chembl_id"] = df4["p_inchikey"].apply(get_chembl_id)
df4["chembl_mismatch"] = df4["p_chembl_id"].apply(lambda x: True if x == "error" else False )
df4.to_csv("../csv_files/adding_chembl_to_actives_.csv")

## Generating final files for database

In [ ]:
# drug_products.csv

drug_product_df = pd.read_csv("../csv_files/final_master_df.csv", index_col=0)
drug_product_df = drug_product_df[['Medicine name',
       'Product number', 'Patient safety',
       'Authorisation status', 'ATC code', 'Additional monitoring', 'Generic',
       'Conditional approval', 'Exceptional circumstances',
       'Accelerated assessment', 'Orphan medicine',
       'Marketing authorisation date',
       'Marketing authorisation holder/company name',
       'Human pharmacotherapeutic group',  'Condition / indication', 'URL', 'Dosage Form', 'Route']]
drug_product_df = drug_product_df.reset_index()
drug_product_df.rename(columns={
    'Column2':"product_id",
    'Medicine name':"product_name",
    'Product number':"ema_number", 
    'Patient safety':"patient_safety",
    'Authorisation status':"authorisation_status", 
    'ATC code':"atc_code", 
    'Additional monitoring':"additional", 
    'Generic':"generic",
    'Conditional approval':"conditional", 
    'Exceptional circumstances':"exceptional",
    'Accelerated assessment':"accelerated", 
    'Orphan medicine':"orphan",
    'Marketing authorisation date':"authorisation_date",
    'Marketing authorisation holder/company name':"authorisation_holder",
    'Human pharmacotherapeutic group':"therapeutic_group",  
    'Condition / indication':"indication",
    'URL':"url",
    'Dosage Form':"dosage_form", 
    'Route':"route"}, inplace=True)
drug_product_df.replace({'yes': True, 'no': False}, inplace=True)
drug_product_df.to_csv("../pharmacoinformatics_database/drug_products.csv", index=False)
drug_product_df.dtypes

In [ ]:
# parent_drugs.csv

active_substances = pd.read_csv("../csv_files/adding_chembl_to_actives.csv", index_col=0)
intravenous_pharmacokinetics = pd.read_csv("../csv_files/lombardo_dataset.csv", encoding="ANSI")
def get_inchikey(smiles):
    try:
        return Chem.MolToInchiKey(Chem.MolFromSmiles(smiles)) # tried getting parent and does not improve matching
    except:
        return "error"
intravenous_pharmacokinetics["Inchikey"] = intravenous_pharmacokinetics["SMILES"].apply(get_inchikey)
intravenous_pharmacokinetics = intravenous_pharmacokinetics[["Inchikey",
                                                             "human_VDss_L_per_kg",
                                                             "human_CL_mL_min_kg",	
                                                             "fraction_unbound_plasma",	
                                                             "MRT_h",
                                                             "terminal_half_life_h"]]
intravenous_pharmacokinetics.rename(columns={"Inchikey":"p_inchikey",
                                            "human_VDss_L_per_kg":"vdss",
                                            "human_CL_mL_min_kg":"clearance",	
                                            "MRT_h":"mrt",
                                            "terminal_half_life_h":"terminal_half_life"}, inplace=True)
active_substances = active_substances[["p_smiles",
                                       "p_inchi",
                                       "p_chembl_id",
                                       "p_inchikey"]]
active_substances.rename(columns={"Substance_Name":"original_substance_name"}, inplace=True)
active_substances.drop_duplicates(subset="p_inchikey", inplace=True)
active_substances_final = pd.merge(active_substances, intravenous_pharmacokinetics, how="left", on=["p_inchikey"])
active_substances_final.drop_duplicates(subset="p_inchikey", keep="last", inplace=True) # Two entries for repaglinide, keeping second as more data
active_substances_final = active_substances_final.reset_index()
active_substances_final.rename(columns={"index":"parent_drug_id"}, inplace=True)
active_substances_final.to_csv("../pharmacoinformatics_database/parent_drugs.csv", index=False)
active_substances_final.dtypes

In [ ]:
# formulated_drugs.csv 

formulated_drugs_df = pd.read_csv("../csv_files/final_master_df.csv", index_col=0)
drug_structures = pd.read_csv("../csv_files/adding_chembl_to_actives.csv")

formulated_drugs_df = formulated_drugs_df[["actives_in_dosage_form", "actives_by_dose", "dose", "Fa1", "Fa2", "Fa3", "Fa4", "F1", "F2", "F3", "F4", "Tmax1", "Tmax2", "Tmax3", "Tmax4", "Notes"]]
formulated_drugs_df["actives_in_dosage_form"] = formulated_drugs_df["actives_in_dosage_form"].apply(lambda x: x.upper().strip().split(","))
formulated_drugs_df["actives_by_dose"] = formulated_drugs_df["actives_by_dose"].apply(lambda x: x.upper().strip().split(","))
formulated_drugs_df["dose"] = formulated_drugs_df["dose"].apply(lambda x: x.upper().strip().split("/"))
formulated_drugs_df["Fa"] = formulated_drugs_df.apply(lambda row: [row["Fa1"], row["Fa2"], row["Fa3"], row["Fa4"]], axis=1)
formulated_drugs_df["F"] = formulated_drugs_df.apply(lambda row: [row["F1"], row["F2"], row["F3"], row["F4"]], axis=1)
formulated_drugs_df["Tmax"] = formulated_drugs_df.apply(lambda row: [row["Tmax1"], row["Tmax2"], row["Tmax3"], row["Tmax4"]], axis=1)
formulated_drugs_df.drop(["Fa1", "Fa2", "Fa3", "Fa4", "F1", "F2", "F3", "F4", "Tmax1", "Tmax2", "Tmax3", "Tmax4"], axis=1, inplace=True)

def prepare_for_explode(row):
    length = len(row["actives_in_dosage_form"])
    row["Fa"] = row["Fa"][:length]
    row["F"] = row["F"][:length]
    row["Tmax"] = row["Tmax"][:length]
    return row
formulated_drugs_df = formulated_drugs_df.apply(prepare_for_explode, axis=1)
formulated_drugs_df = formulated_drugs_df.explode(["actives_in_dosage_form", "actives_by_dose", "dose", "Fa", "F", "Tmax"])
formulated_drugs_df["dose_value"] = formulated_drugs_df["dose"].str.extract(r"(\d+\.?\d*)").astype(float)
formulated_drugs_df["dose_unit"] = formulated_drugs_df["dose"].str.extract(r"(\d+\.?\d*\s*)(.*)")[1].str.strip()
formulated_drugs_df.drop(columns=["dose"], inplace=True)
formulated_drugs_df = formulated_drugs_df.reset_index()
formulated_drugs_df.rename(columns={"Column2":"product_id"}, inplace=True)
formulated_drugs_df["actives_in_dosage_form"] = formulated_drugs_df["actives_in_dosage_form"].str.strip()
drug_structures = drug_structures[["active_PSS",
                                   "Inchikey",
                                   "pss_inchi",
                                   "IsomericSMILES",
                                   "ps_smiles",
                                   "ps_num_drugs",
                                   "p_inchikey"]]
drug_structures.rename(columns={
    "active_PSS":"actives_in_dosage_form",
    "Inchikey":"pss_inchikey",
    "IsomericSMILES":"pss_smiles"
}, inplace=True)

drug_structures.drop_duplicates(subset="actives_in_dosage_form", keep="first", inplace=True)
final_formulated_drugs = pd.merge(formulated_drugs_df, drug_structures, how="left", on="actives_in_dosage_form")
mapping_df = pd.read_csv("../pharmacoinformatics_database/parent_drugs.csv")
mapping_df = mapping_df[["p_inchikey", "parent_drug_id"]]
mapping = mapping_df.set_index("p_inchikey")["parent_drug_id"].to_dict()
final_formulated_drugs["parent_drug_id"] = final_formulated_drugs["p_inchikey"].map(mapping)
final_formulated_drugs.drop(columns=["p_inchikey"], inplace=True)
final_formulated_drugs[["Fa", "F", "Tmax"]] = final_formulated_drugs[["Fa", "F", "Tmax"]].replace("Blank", np.nan)
final_formulated_drugs[["Fa", "F", "Tmax"]] = final_formulated_drugs[["Fa", "F", "Tmax"]].astype(float)
final_formulated_drugs.rename(columns={"Fa":"fa", 
                                       "F":"f", 
                                       "Tmax":"tmax",
                                       "Notes":"notes"}, inplace=True)
final_formulated_drugs.to_csv("../csv_files/formulated_drugs_.csv", index=False) #curation needed to remove notes
final_formulated_drugs_curated = pd.read_csv("../pharmacoinformatics_database/formulated_drugs.csv")
final_formulated_drugs_curated.dtypes

In [ ]:
# formulations.csv DO NOT CHANGE

excipients_df = pd.read_csv("../csv_files/final_master_df.csv", index_col=0)
excipients_df = excipients_df[["Excipients_Final"]]
excipients_df.rename(columns={"Excipients_Final":"excipient"}, inplace=True)
excipients_df["excipient"] = excipients_df["excipient"].apply(ast.literal_eval)
excipients_df = excipients_df.explode("excipient")
excipients_list = excipients_df["excipient"].dropna().unique()
excipient_dict = {excipient: int(num + 1) for num, excipient in enumerate(excipients_list)}
excipients_df["excipient_id"] = excipients_df["excipient"].map(excipient_dict).fillna(0)
excipients_df["excipient_id"] = excipients_df["excipient_id"].astype(int)
excipients_df = excipients_df.reset_index()
excipients_df.rename(columns={"Column2":"product_id"}, inplace=True)
excipients_df.to_csv("../pharmacoinformatics_database/formulations.csv", index_label="product_id")
excipients_df.dtypes

In [ ]:
# excipients.csv
#import time
excip_chem_df = pd.DataFrame(list(excipient_dict.items()), columns=["excipient_name", "excipient_id"])
# Apply the function to each drug in the dataframe
excip_chem_df["inchi"] = excip_chem_df["excipient_name"].apply(get_pubchem_for_excipient)
excip_chem_df.to_csv("../csv_files/excipients_.csv")

In [ ]:
# Getting ChEMBL and PubChem ID from curated excipients

excipients_with_inchi = pd.read_csv("../csv_files/excipients_curated.csv", index_col=0)
excipients_with_inchi["excipient_inchikey"] = excipients_with_inchi["inchi"].apply(get_inchikey_from_inchi)
excipients_with_inchi["excipient_chembl_id"] = excipients_with_inchi["excipient_inchikey"].apply(constructiontools.get_chembl_id)
excipients_with_inchi.replace("error", np.nan, inplace=True)
excipients_with_inchi.rename(columns={"inchi":"excipient_inchi"}, inplace=True)
excipients_with_inchi["excipient_pubchem_cid"] = excipients_with_inchi["excipient_inchikey"].apply(get_pubchem_cid)
excipients_with_inchi.to_csv("../pharmacoinformatics_database/excipients.csv", index=False)

In [ ]:
# Automated construction of data dictionary

list_for_data_dict = [
    (drug_product_df, "drug_products.csv"),
    (active_substances_final, "parent_drugs.csv"),
    (final_formulated_drugs_curated, "formulated_drugs.csv"),
    (excipients_df, "formulations.csv"),
    (excipients_with_inchi, "excipients.csv")
]

create_combined_data_dictionary(list_for_data_dict).to_csv("../pharmacoinformatics_database/data_dictionary.csv", index=False)